In [ ]:
%%configure -f 
{
    "conf":
    {
        "spark.app.name":"Bhavesh_Relevancy_notebook",
        "spark.yarn.queue": "default",
        "spark.jars": "/apps/Jars/obelisk-retail-legos.jar,/apps/Jars/mysql-connector-java-5.1.40.jar",
        "spark.driver.extraJavaOptions": "-Dscala.color",
        "spark.driver.memory": "2g",
        "spark.executor.memory": "2g",
        "spark.executor.instances": "4"
    }
}

In [ ]:
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._

In [ ]:
val exp = "neural_network_new"

In [ ]:
def metricsShow(exp: String): Unit = {
    var pl = CSVToDF.getDF(s"/data/Archive/bhavesh/inventoryPrediction/experiments/exp_${exp}/metrics/productLevelModelForEachBrick/productLevel", inferSchema=true).orderBy("model", "similargrouplevel").drop(DATE).persist()
    pl.printSchema

    var show = pl.orderBy("metric").groupBy("model", "metric").pivot("similargrouplevel").agg(first("value")).orderBy("model", "metric").withColumn("final", concat(lit("Men Shirts: "), col("830216013"), lit("\n"), lit("Women Kurtas: "), col("830303011"))).groupBy("model").pivot("metric").agg(first("final") as "final").persist()
    show.orderBy("model").show(false)
    // show.orderBy("model").drop("model").show(false)
    DFToCSV.putDF("/data/Archive/plp/bhavesh/temp/ip/pl_show", show.orderBy("model").coalesce(1))
    pl.unpersist()

    var ql = CSVToDF.getDF(s"/data/Archive/bhavesh/inventoryPrediction/experiments/exp_${exp}/metrics/productLevelModelForEachBrick/queryLevel", inferSchema=true).orderBy("model", "similargrouplevel").drop(DATE).persist()
    ql.printSchema

    var show = ql.orderBy("metric").groupBy("model", "metric").pivot("similargrouplevel").agg(first("value")).orderBy("model", "metric").withColumn("final", concat(lit("Men Shirts: "), col("830216013"), lit("\n"), lit("Women Kurtas: "), col("830303011"))).groupBy("model").pivot("metric").agg(first("final") as "final").persist()
    show.orderBy("model").show(false)
    // show.orderBy("model").drop("model").show(false)
    DFToCSV.putDF("/data/Archive/plp/bhavesh/temp/ip/ql_show", show.orderBy("model").coalesce(1))
    ql.unpersist()
}
